In [1]:
from pyspark.sql import SparkSession

In [119]:
from pyspark.sql.functions import when

In [2]:
spark = SparkSession.builder\
        .master("local[6]")\
        .appName('task1')\
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/31 12:58:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
json_file = './assignment_data.jsonl'
df = spark.read.json(json_file)

In [6]:
print(df.schema)
df.show()

StructType([StructField('country', StringType(), True), StructField('event', StringType(), True), StructField('player_id', StringType(), True), StructField('session_id', StringType(), True), StructField('ts', StringType(), True)])
+-------+-----+--------------------+--------------------+-------------------+
|country|event|           player_id|          session_id|                 ts|
+-------+-----+--------------------+--------------------+-------------------+
|     PK|start|d6313e1fb7d247a6a...|674606b1-2270-428...|2016-11-22T20:40:50|
|   null|  end|20ac16ebb30a47708...|16ca9d01-d240-452...|2016-11-18T06:24:50|
|   null|  end|318e22b061b54042b...|5f933591-8cd5-414...|2016-11-16T18:01:37|
|   null|  end|29bb390d9b1b4b4b9...|ef939180-692a-484...|2016-11-13T10:38:09|
|     GH|start|a477ecabc3cc455cb...|4c55263e-66b2-481...|2016-11-29T19:31:43|
|     SK|start|1ec36a67785046b3b...|3346a60a-0989-404...|2016-11-16T05:36:16|
|     DE|start|9595af0063e94cb8a...|06830030-d091-428...|2016-11-07

In [9]:
df.columns

['country', 'event', 'player_id', 'session_id', 'ts']

In [10]:
df.count()

1001169

In [11]:
df_sessions = df.select('session_id')

In [13]:
df_sessions.show()

+--------------------+
|          session_id|
+--------------------+
|674606b1-2270-428...|
|16ca9d01-d240-452...|
|5f933591-8cd5-414...|
|ef939180-692a-484...|
|4c55263e-66b2-481...|
|3346a60a-0989-404...|
|06830030-d091-428...|
|a78a4889-4bcf-45a...|
|dd223ea6-0e6b-4dd...|
|3015bf71-4b28-4c9...|
|1d9e6d4a-e1c0-402...|
|c650360a-3a6c-495...|
|b20c6b98-7f18-4ba...|
|c522efe9-847a-47b...|
|58dd0153-026d-455...|
|94483e5b-bebb-46c...|
|4f7e416d-fc11-4c3...|
|75cbd945-817a-40a...|
|9d73326f-800a-466...|
|cd0f2f42-3ab2-450...|
+--------------------+
only showing top 20 rows



In [15]:
df_sessions.distinct().count()

500587

In [ ]:
#2

In [25]:
df_get_player_contry = df.select('player_id', 'country')

In [27]:
df_get_player_contry = df_get_player_contry.filter(df_get_player_contry['country'].isNotNull()).distinct()

In [29]:
df_get_player_contry.show()

+--------------------+-------+
|           player_id|country|
+--------------------+-------+
|4711ec7f3d21415ca...|     PK|
|a7be5d00673a4ef7b...|     US|
|95f025dcde354b77b...|     SO|
|5543196eab7e4d11b...|     PM|
|8371f1125c9b4217a...|     PM|
|49bee45c9c6f4fde9...|     NO|
|8a554ef458c64f9b9...|     MO|
|c7b0293eddab49dab...|     KW|
|a1517653eaf442789...|     BW|
|da88ec56a16949849...|     MP|
|e729f530c8cc410c9...|     ML|
|7a3f81604d114f4c9...|     SK|
|93acf3f8f4c7414c9...|     IR|
|4c61b61e7af446f69...|     LA|
|0ce80e676b0c4889b...|     DO|
|f5da2bbefbd6468bb...|     SI|
|82843986473144f0b...|     PA|
|d22e3caf64914095b...|     MP|
|b8a790ea6a664b109...|     SO|
|265a931b890a4a1d9...|     MD|
+--------------------+-------+
only showing top 20 rows



In [140]:
end_df = df.select('player_id', 'country','event').filter(df['event'] == 'end')

In [141]:
end_df.show()

+--------------------+-------+-----+
|           player_id|country|event|
+--------------------+-------+-----+
|20ac16ebb30a47708...|   null|  end|
|318e22b061b54042b...|   null|  end|
|29bb390d9b1b4b4b9...|   null|  end|
|8d0e3cd4a25d4a089...|   null|  end|
|e59f1fa31e144fd8b...|   null|  end|
|fd8a1e9fff25471da...|   null|  end|
|7eaf41e8264946cf9...|   null|  end|
|25752b769f5744e98...|   null|  end|
|fc287863d671446db...|   null|  end|
|c408369703f940bb9...|   null|  end|
|33e5f534cd074991a...|   null|  end|
|2e43f27713e14e57a...|   null|  end|
|ca00e428db7541f29...|   null|  end|
|c6cfdef84b264d7da...|   null|  end|
|24a008c2d5084f478...|   null|  end|
|2c240017adcf4bff9...|   null|  end|
|3aa403a22f5a40cf8...|   null|  end|
|5a57acfbd4f34d45b...|   null|  end|
|0bd5fdd919134c77b...|   null|  end|
|0612127e93ca4ce8b...|   null|  end|
+--------------------+-------+-----+
only showing top 20 rows



In [145]:
end_df2 = end_df.withColumn("country", df_get_player_contry.collect()[0][1])

TypeError: col should be Column

In [ ]:
#Сколько сессий завершает каждый игрок? Выведите 10 лучших результатов, отсортированных по
#count(DESC) и player_id(ASC).

In [59]:
df_get_player = df.select('player_id', 'event')

In [60]:
df_get_player = df_get_player.filter(df_get_player['event'] == 'end')

In [61]:
df_players_end = df_get_player.groupBy('player_id').count()

In [42]:
df_players_end.show()

+--------------------+-----+
|           player_id|count|
+--------------------+-----+
|1e61e4785d764b92b...|   84|
|05e73a69c41943498...|   50|
|c167eb777aba4319b...|   86|
|465f81dfdee345669...|   66|
|522a28a053a44af69...|   92|
|3f282f08b347481b8...|   74|
|cd5742d5ad5247a79...|    6|
|3e888630d54e4dde9...|   84|
|3aa78890e9454b46b...|   40|
|ec0a0483fddd4ebfa...|   98|
|fffa44da04a24d799...|   54|
|d40ae511199c42ef8...|   60|
|f147eebba02a4da3b...|   84|
|37370d0139544d1ca...|  100|
|7f51dffc2bd34e80b...|   54|
|42b734c177914718a...|   20|
|e8f36a8efded42bfb...|  100|
|f5c559197ab34adfa...|    6|
|293da457fd584ea18...|   68|
|dce1ebcf453e48a38...|   82|
+--------------------+-----+
only showing top 20 rows



In [50]:
from pyspark.sql.functions import col
from pyspark.sql.functions import desc

In [63]:
df_players_end.sort(col("count").desc(),col("player_id")).show(10)

+--------------------+-----+
|           player_id|count|
+--------------------+-----+
|0052e4cfaf494af29...|   50|
|008c31d6d87d4a72a...|   50|
|019f6864a44e4cd1a...|   50|
|01d1f31c3c864538a...|   50|
|0259292bb77e4aaba...|   50|
|04318f09e6e7434bb...|   50|
|060243602575481db...|   50|
|067b20022f2040b0a...|   50|
|072d69b0074349689...|   50|
|08037331842341b2b...|   50|
+--------------------+-----+
only showing top 10 rows



In [ ]:
#В какой стране в 2016 году было начато наибольшее количество сеансов?

In [80]:
starts = df.select('country','event').filter(df['event'] == 'start')

In [81]:
starts.show()

+-------+-----+
|country|event|
+-------+-----+
|     PK|start|
|     GH|start|
|     SK|start|
|     DE|start|
|     FM|start|
|     SC|start|
|     KG|start|
|     EE|start|
|     GS|start|
|     BN|start|
|     NR|start|
|     GY|start|
|     SV|start|
|     IS|start|
|     NA|start|
|     UA|start|
|     GF|start|
|     BE|start|
|     MQ|start|
|     PF|start|
+-------+-----+
only showing top 20 rows



In [82]:
starts.count()

500584

In [83]:
starts = starts.groupby('country').count()

In [84]:
starts.show()

+-------+-----+
|country|count|
+-------+-----+
|     DZ| 1953|
|     LT| 2276|
|     MM| 1996|
|     TC| 2206|
|     SC| 1774|
|     PM| 2112|
|     AZ| 2670|
|     FI| 2349|
|     UA| 2141|
|     KI| 1662|
|     ZM| 1680|
|     RO| 2550|
|     SL| 2202|
|     LA| 2402|
|     NL| 2247|
|     SB| 1626|
|     BW| 2168|
|     MN| 2365|
|     BS| 2239|
|     PL| 2119|
+-------+-----+
only showing top 20 rows



In [87]:
starts.sort(col("count").desc()).show(1)

+-------+-----+
|country|count|
+-------+-----+
|     IT| 2839|
+-------+-----+
only showing top 1 row

